In [ ]:
import pandas as pd
import json
from pandas.io.json import json_normalize

In [ ]:
"""Get all boot reports for a job and a specified kernel"""
""" Sort boards by average boot time """
""" Sort boards by boot success percentage """
import requests
from urlparse import urljoin
import kernelci_api_key

BACKEND_URL = "http://api.kernelci.org"

def invoke():
    headers = {
        "Authorization": kernelci_api_key.getkernelcikey()
    }
    params = {
        "job":"mainline",
        "date_range" : 30
    }
    url = urljoin(BACKEND_URL, "/boot")
    response = requests.get(url, params=params, headers=headers)
    return response.content

In [ ]:
content = invoke()
content


In [ ]:
contentjs = json.loads(content)
contentjs['result']

In [ ]:
df = json_normalize(contentjs['result'])

In [ ]:
df['created_on.$date'] = pd.to_datetime(df['created_on.$date'],unit='ms')
df['created_on.$date'].head()

In [ ]:
df = df[df.arch=='arm']
df.head()

In [ ]:
## Understanding Boot times for each board
boottimedf = df.groupby(df.board)
boottimedf.mean().sort('time.$date',axis=0, ascending=False)

In [ ]:
## What are the unique entries in this table
dfd = df.describe(include='all')
dfd.loc['unique',:]

In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
dfBestBootBoards=df[[u'_id.$oid', u'arch', u'board', u'board_instance',u'status']]

In [ ]:
dfBestBootBoards.head()

In [ ]:
dfBestBootBoards.status.value_counts()

In [ ]:
dfBestBootBoards = dfBestBootBoards[(dfBestBootBoards.status!='OFFLINE')]
dfBestBootBoards.status.value_counts()

In [ ]:
dfBestBootBoards.head()

In [ ]:
dfBestBootBoards['status_fl'] = dfBestBootBoards.status.apply(lambda x: 1.0 if (x=='PASS') else 0.0)

In [ ]:
dfBestBootBoards.head()

In [ ]:
dfBestBootBoards.groupby('board').mean().sort('status_fl')

In [ ]:
dfBestBootBoards.board.value_counts()